In [15]:
import sys
import os
import pandas as pd
from skyfield.api import Topos, load
from datetime import datetime, timezone
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import fct_space 

## Step 1 : TLE generation

In [16]:
raan = 120.0  # RAAN en degrés
inclination = 97.5  # Inclinaison en degrés
altitude = 500.0  # Altitude en km
date_time = datetime(2024, 1, 1, 0, 0, 0)  # Date/Time (UTC)

# --- other params
arg_perigee = 0 # Argument du périgée en degrés
sma = fct_space.Rt + altitude  # Semi-major axis en km pour une orbite basse
eccentricity = 0.0  # Excentricité (typique pour cubesat en LEO)

# Générer TLEb

tle_line_1, tle_line_2 = fct_space.generate_tle(
    raan, inclination, arg_perigee, sma, eccentricity, date_time
)

## Step 2 : Gather launch date

In [17]:
link = 'https://destination-orbite.net/astronautique/lancements/recapitulatif/lancements-orbitaux-2024'

In [18]:
data=pd.read_html(link)
df = data[0]

In [19]:
df['DATE HEURE'] = pd.to_datetime(df['DATE HEURE'], format='mixed').dt.tz_localize(None).dt.floor('S')
df = df.set_index('DATE HEURE').sort_index()

/tmp/ipykernel_45489/633402201.py:1: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



## Step 3 : Test on a special launch site

In [20]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count",  x=df[df['COSPAR PAYS'].str.contains('Chine')]['CENTRE SPATIAL PAS DE TIR']))
fig.update_layout(title_text='Lancements de satellites chinois en 2024')
fig.show()

**test for Xichang Space Center**

In [21]:
site = 'Xichang Space Center'
# coordinates for Xichang Space Center
xsp_lat = 28.1446
xsp_long = 102.0136
xsp_height = 0.0
print(f"Test for Xichang Space Center coordinates: {xsp_lat}, {xsp_long}, {xsp_height}")

Test for Xichang Space Center coordinates: 28.1446, 102.0136, 0.0


In [22]:
df_chine_selected = df[df['CENTRE SPATIAL PAS DE TIR'].str.contains(site)]

In [23]:
nb_overpass_site = 0
for date in df_chine_selected.index:
    elev_date = fct_space.est_visible(tle_line_1, tle_line_2, xsp_lat, xsp_long, xsp_height, date)
    print(f'for the laught at {date} on {site}, satellite was at {elev_date:.1f}°')
    if elev_date > 0:
        nb_overpass_site += 1


for the laught at 2024-01-08 13:14:00 on Xichang Space Center, satellite was at -37.9°
for the laught at 2024-02-02 23:37:00 on Xichang Space Center, satellite was at -55.9°
for the laught at 2024-02-04 22:56:00 on Xichang Space Center, satellite was at -43.4°
for the laught at 2024-02-29 13:03:00 on Xichang Space Center, satellite was at -42.8°
for the laught at 2024-03-09 01:22:00 on Xichang Space Center, satellite was at -58.8°
for the laught at 2024-03-12 05:56:00 on Xichang Space Center, satellite was at -18.5°
for the laught at 2024-03-13 12:51:00 on Xichang Space Center, satellite was at -46.3°
for the laught at 2024-04-12 04:46:00 on Xichang Space Center, satellite was at -40.8°
for the laught at 2024-04-20 23:45:00 on Xichang Space Center, satellite was at -62.9°
for the laught at 2024-05-30 12:12:00 on Xichang Space Center, satellite was at -45.8°
for the laught at 2024-06-22 07:00:00 on Xichang Space Center, satellite was at -28.5°
for the laught at 2024-08-16 07:35:00 on Xi

In [24]:
print(f'Number of overpass for {site} in 2024: {nb_overpass_site}')
print(f'Probability of mission success for {site} in 2024: {nb_overpass_site/len(df_chine_selected)}')

Number of overpass for Xichang Space Center in 2024: 0
Probability of mission success for Xichang Space Center in 2024: 0.0


## Step 4 : Test for all orbits

In [25]:
raans = []
inclinations = []
altitudes = []
probabilities = []


for raan_ in range(0, 360, 10):
    for inclination_ in range(50, 100+10, 5):
        for altitude_ in range(400, 600+50, 20):
            sma_ = fct_space.Rt + altitude_
            tle_line_1, tle_line_2 = fct_space.generate_tle(
                raan_, inclination_, arg_perigee, sma_, eccentricity, date_time
            )
            # print(f'TLE for RAAN: {raan_}, Inclination: {inclination_}, Altitude: {altitude_}')
            nb_overpass_site = 0
            for date in df_chine_selected.index:
                elev_date = fct_space.est_visible(tle_line_1, tle_line_2, xsp_lat, xsp_long, xsp_height, date)
                # print(f'for the laught at {date} on {site}, satellite was at {elev_date:.1f}°')
                if elev_date > 0:
                    print(f'for the laught at {date} on {site}, satellite was at {elev_date:.1f}°')
                    nb_overpass_site += 1
            raans.append(raan_)
            inclinations.append(inclination_)
            altitudes.append(altitude_)
            probabilities.append(nb_overpass_site/len(df_chine_selected))
df_prob = pd.DataFrame([raans, inclinations, altitudes, probabilities], index=['raan', 'inclinaison', 'altitude', 'probabilities']).T

In [26]:
df_prob

,raan,inclinaison,altitude,probabilities
0,0.0,50.0,400.0,0.0
1,0.0,50.0,420.0,0.0
2,0.0,50.0,440.0,0.0
3,0.0,50.0,460.0,0.0
4,0.0,50.0,480.0,0.0
...,...,...,...,...
5611,350.0,105.0,560.0,0.0
5612,350.0,105.0,580.0,0.0
5613,350.0,105.0,600.0,0.0
5614,350.0,105.0,620.0,0.0


In [27]:
df_prob[df_prob['probabilities'] > 0]

,raan,inclinaison,altitude,probabilities
